In [2]:
import os
import shutil
import xml.etree.ElementTree as ET
from ultralytics import YOLO
import random
import cv2
import pytesseract
import torch
import easyocr
import re
import time
import json
import re
import numpy as np
from collections import defaultdict, deque



In [2]:
import requests
import os
from requests_toolbelt.multipart.encoder import MultipartEncoder

def get_def_headers():
    return {
        "X-API-KEY": "123456"
    }

def get_base_api():
    return 'http://ec2-54-87-52-160.compute-1.amazonaws.com'


# Camera id use for identify location of their cameras
# please use these id below for testing:
# ad7de137-9287-402a-8b70-53684d96c88f: Future park, Rangsit, Thanyaburi, Prachatipat, Pathum Thani
# 2ec15a48-c819-494c-a807-5c0f41ebaf36: BTS Asok, Klongtoey Noei, Wattana, Bangkok
# 0e76998d-0590-4679-924a-049f92ab0b81: Lotus Laksi, Bangkhen, Anusawaree, Bangkok


# Send Notify API
def send_notify(license_plate: str, camera_id: str, upload_id: str):
    notify_api = get_base_api() + "/notify/v1/send"
    notify_json = {
        'licensePlate': license_plate,
        'cameraId': camera_id,
        'uploadId': upload_id
    }
    response = requests.post(notify_api, json=notify_json, headers=get_def_headers())
    return response.json()

# Upload Image API
def upload_image(image_path):
    upload_api = get_base_api() + "/media/v1/upload/image"
    filename = os.path.basename(image_path)

    with open(image_path, 'rb') as img_file:
        multipart_data = MultipartEncoder(
            fields={'image': (filename, img_file, 'image/jpeg')}
        )
        headers = get_def_headers()
        headers['Content-Type'] = multipart_data.content_type
        
        response = requests.post(upload_api, data=multipart_data, headers=headers)
    return response.json()

In [3]:
# Example Send Notify without Image
no_img_license_plate = "7กญ 3603 กรุงเทพมหานคร"
no_img_camera = "ad7de137-9287-402a-8b70-53684d96c88f"
response = send_notify(no_img_license_plate, no_img_camera, '')
print("Send Notify without Image response:", response)

Send Notify without Image response: {'notifyId': 'c2a1390c-45e3-414b-a6fb-a530a0409b6d', 'status': 'PENDING'}


In [4]:
# Example Send Upload Image
img_path = './runs/detect/predict4/car02.jpg'
upload_response = upload_image(img_path)
print("Send Upload response:", upload_response)

# Example Send Notify with Image
img_license_plate = '9กด 1881 กรุงเทพมหานคร'
img_camera = 'ad7de137-9287-402a-8b70-53684d96c88f'
img_upload = upload_response['uploadId']
notify_response = send_notify(img_license_plate, img_camera, img_upload)
print("Send Notify with Image response:", response)

Send Upload response: {'uploadId': '71bfe9c0-204f-4011-a1ed-6e81be8f386f', 'fileId': '5ff3a40a-513d-47c2-9d60-d0afacbf0e6f', 'filePath': '71bfe9c0-204f-4011-a1ed-6e81be8f386f/5ff3a40a-513d-47c2-9d60-d0afacbf0e6f.jpg', 'contentType': 'jpg', 'status': 'SUCCESS'}
Send Notify with Image response: {'notifyId': 'c2a1390c-45e3-414b-a6fb-a530a0409b6d', 'status': 'PENDING'}


In [ ]:
# -------------------------------
# ใช้สำหรับแบ่ง dataset เป็น train/val/test
# -------------------------------
images_dir = "data/images_all"      # โฟลเดอร์เก็บภาพทั้งหมด (เช่น D:\yolo\Project_2\data\images_all)
labels_dir = "data/labels_all"      # โฟลเดอร์เก็บ labels ทั้งหมด (เช่น D:\yolo\Project_2\data\labels_all)

output_base = "datasets/YOLO"       # โฟลเดอร์สำหรับ train/val/test
split_ratio = (0.7, 0.2, 0.1)       # train, val, test

# -------------------------------
# สร้างโฟลเดอร์ input ถ้าไม่มี
# -------------------------------
if not os.path.exists(images_dir):
    os.makedirs(images_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{images_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ภาพก่อน")

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{labels_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ labels ก่อน")

# -------------------------------
# สร้างโฟลเดอร์ output (images + labels + split)
# -------------------------------
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_base, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_base, "labels", split), exist_ok=True)

# -------------------------------
# รวมรายชื่อไฟล์ภาพ
# -------------------------------
images = [f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))]

if not images:
    print(f"⚠️ ไม่พบไฟล์ภาพใน '{images_dir}' → กรุณาใส่ภาพก่อน")
    exit()

import random
random.shuffle(images)

n_total = len(images)
n_train = int(split_ratio[0] * n_total)
n_val = int(split_ratio[1] * n_total)
n_test = n_total - n_train - n_val

print(f"พบทั้งหมด {n_total} รูป → Train={n_train}, Val={n_val}, Test={n_test}")

# -------------------------------
# ฟังก์ชันย้ายไฟล์
# -------------------------------
import shutil
def move_files(file_list, split):
    for img_file in file_list:
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(output_base, "images", split, img_file)

        # path ของ label (ชื่อเดียวกันแต่ .txt)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(output_base, "labels", split, label_file)

        # copy ภาพ
        shutil.copy(src_img, dst_img)

        # copy label ถ้ามี
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)
        else:
            print(f"⚠️ ไม่มี label สำหรับ {img_file}")

# -------------------------------
# แบ่ง dataset
# -------------------------------
move_files(images[:n_train], "train")
move_files(images[n_train:n_train+n_val], "val")
move_files(images[n_train+n_val:], "test")

print("✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน:", output_base)


พบทั้งหมด 1402 รูป → Train=981, Val=280, Test=141
✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน: datasets/YOLO


In [ ]:
images_dir = "cartest"  
torch.cuda.empty_cache()


data_yaml  = "datasets/YOLO/data.yaml"
# -------------------------------


# เลือกโมเดล
model = YOLO("yolov8s.pt")  # small model, VRAM 8GB ยังรองรับ

# Train
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=560,       # ขนาดภาพใหญ่ขึ้น
    batch=8,        
    name="plate_detector",
    augment=True,    # เพิ่มความแม่น
    device=0 ,         # ใช้ GPU
    half=True     # FP16 ลด VRAM
    
)

# ทดสอบโมเดล
results = model.predict(
    source=images_dir,
    save=True,
    conf=0.5
)
print("✅ Prediction finished, check runs/detect/plate_detector/")



New https://pypi.org/project/ultralytics/8.3.206 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.201  Python-3.10.18 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)


AttributeError: module 'torch._C' has no attribute '_has_mps'

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Parking-watch simulator:
- รันกล้องต่อเนื่อง
- ตรวจหา plate ทุก 10 วินาที
- ถ้าไม่มีรถ -> ไม่มีอะไรเกิดขึ้น
- ถ้ามีทะเบียน -> เก็บเป็น target
- ถ้าเจอทะเบียนเดิมต่อเนื่องและเวลาตั้งแต่เจอครั้งแรก >= 120s -> แคปภาพทั้งคัน + print บันทึก
- ถ้าเจอทะเบียนใหม่ (ไม่ตรงกับ target) -> รีเซ็ต target ไปหาใหม่
- พิมพ์สถานะทั้งหมด (no DB, no network)
"""
import os
import re
import cv2
import time
import numpy as np
import torch
import difflib
from ultralytics import YOLO
from datetime import datetime

# ---------------------------- fast_alpr import ----------------------------
try:
    from fast_alpr.alpr import ALPR, BaseOCR, OcrResult
except Exception as e:
    raise RuntimeError(f"❌ fast_alpr import failed: {e}\nInstall with: pip install fast-alpr")

# ---------------------------- จังหวัดไทย ----------------------------
thai_provinces = [
    "กรุงเทพมหานคร", "กระบี่", "กาญจนบุรี", "กาฬสินธุ์", "กำแพงเพชร",
    "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท",
    "ชัยภูมิ", "ชุมพร", "เชียงราย", "เชียงใหม่", "ตรัง",
    "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม",
    "นครราชสีมา", "นครศรีธรรมราช", "นครสวรรค์", "นนทบุรี", "นราธิวาส",
    "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์",
    "ปราจีนบุรี", "ปัตตานี", "พระนครศรีอยุธยา", "พังงา", "พัทลุง",
    "พิจิตร", "พิษณุโลก", "เพชรบุรี", "เพชรบูรณ์", "แพร่",
    "พะเยา", "ภูเก็ต", "มหาสารคาม", "มุกดาหาร", "แม่ฮ่องสอน",
    "ยโสธร", "ยะลา", "ร้อยเอ็ด", "ระนอง", "ระยอง",
    "ราชบุรี", "ลพบุรี", "ลำปาง", "ลำพูน", "เลย",
    "ศรีสะเกษ", "สกลนคร", "สงขลา", "สตูล", "สมุทรปราการ",
    "สมุทรสงคราม", "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี",
    "สุโขทัย", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "หนองคาย",
    "หนองบัวลำภู", "อ่างทอง", "อำนาจเจริญ", "อุดรธานี", "อุตรดิตถ์",
    "อุทัยธานี", "อุบลราชธานี", "ประเทศไทย"
]

def correct_province(text):
    if not text:
        return None
    match = difflib.get_close_matches(text, thai_provinces, n=1, cutoff=0.3)
    return match[0] if match else None

# ---------------------------- OCR Engine ----------------------------
GPU_AVAILABLE = torch.cuda.is_available()

class EasyOCR_ALPR(BaseOCR):
    def __init__(self, lang_list=['th','en'], min_conf=0.1):
        import easyocr
        self.reader = easyocr.Reader(lang_list, gpu=GPU_AVAILABLE)
        self.min_conf = min_conf
        print(f"✅ EasyOCR_ALPR loaded (GPU={GPU_AVAILABLE})")

    def predict(self, image: np.ndarray):
        # Enhancement simple
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape)==3 else image
        gray = cv2.bilateralFilter(gray, 12, 45, 45)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        gray = clahe.apply(gray)
        ocr_results = self.reader.readtext(gray)
        results = []
        for r in ocr_results:
            bbox, text, conf = r if len(r)==3 else (None, r[1], 1.0)
            if conf >= self.min_conf and text.strip():
                results.append(OcrResult(text=text.strip(), confidence=float(conf)))
        return results

# ---------------------------- helpers (plate extraction + enhance) ----------------------------
def correct_common_thai_ocr_errors(text):
    corrections = {
        "ญณ": "ฌฌ", "ญญ": "ฌฌ", "ญ": "ฌ"
    }
    for wrong, right in corrections.items():
        text = text.replace(wrong, right)
    return text

def upscale_image(img, scale=3):
    h, w = img.shape[:2]
    return cv2.resize(img, (w*scale, h*scale), interpolation=cv2.INTER_LANCZOS4)

def extract_province_from_text(text):
    candidates = re.split(r"[\s\n]+", text)
    for word in candidates:
        match = difflib.get_close_matches(word, thai_provinces, n=1, cutoff=0.25)
        if match:
            return match[0]
    return None

def extract_thai_license_plate(text):
    cleaned = re.sub(r"[\n\r]+", " ", text)
    cleaned = re.sub(r"[^ก-ฮ0-9\s\-\.]", "", text)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    pattern = (
        r"([0-9]{0,2}\s*[ก-ฮ]{1,3}[\s\-\.]*\d{1,4})"     # หมายเลขทะเบียน
        r"[\s\n]*"
        r"(?:จังหวัด)?\s*([ก-ฮ]{2,20})?"                 # จังหวัด (optional)
    )
    matches = re.findall(pattern, cleaned)
    results = []
    for plate, province in matches:
        plate = re.sub(r"[\s\-\.]", "", plate)
        province = province.strip() if province else None
        if province:
            best_match = difflib.get_close_matches(province, thai_provinces, n=1, cutoff=0.25)
            province = best_match[0] if best_match else None
        results.append({"plate": plate, "province": province})
    return results

def enhance_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img.copy()
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(5, 5))
    gray = clahe.apply(gray)
    gray = cv2.bilateralFilter(gray, 6, 60, 60)
    gray = cv2.resize(gray, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, blockSize=55, C=15)
    if np.sum(th == 0) > np.sum(th == 255):
        th = cv2.bitwise_not(th)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    th = cv2.morphologyEx(th, cv2.MORPH_DILATE, kernel, iterations=1)
    return th

def safe_crop(img,x1,y1,x2,y2,pad=5):
    h,w=img.shape[:2]
    x1=max(0,x1-pad)
    y1=max(0,y1-pad)
    x2=min(w,x2+pad)
    y2=min(h,y2+pad)
    return img[y1:y2,x1:x2]

# ---------------------------- Initialize ALPR & YOLO ----------------------------
ocr_engine = EasyOCR_ALPR(lang_list=['th','en'])
alpr = ALPR(ocr=ocr_engine)   # ใช้งาน fast_alpr wrapper ของคุณ
YOLO_WEIGHTS = "runs/detect/plate_detector/weights/best.pt"  # เปลี่ยนให้ถูกต้อง
model = YOLO(YOLO_WEIGHTS)

# ---------------------------- Simulation / monitoring params ----------------------------

source = "demos/hard.mp4" # เปลี่ยนเป็นไฟล์วิดีโอ "car001.mp4" หรือ 0 สำหรับกล้อง
CHECK_INTERVAL = 2.0   # วินาที — ตรวจหา plate ทุก 10 วิ
LONG_STAY_THRESHOLD = 10.0  # วินาที — ถ้าซ้ำเกิน 2 นาที => long stay
OUTPUT_DIR = "output_longstay"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# tracking state (single-target logic ตามที่ขอ)
target_plate = None            # ป้ายที่กำลังติดตาม (string)
target_first_seen = None       # datetime ของการพบครั้งแรกสำหรับ target
target_last_seen = None        # datetime ของการพบล่าสุด (update ทุกครั้งที่เห็น same plate)

# ---------------------------- Main loop ----------------------------
cap = cv2.VideoCapture(source)
last_check = 0.0

print("✅ เริ่มระบบ (ตรวจทุก 10 วินาที). กด q เพื่อออก")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ ไม่สามารถอ่าน frame ได้ (end of video or camera error). หยุดการทำงาน.")
        break

    now = time.time()
    # แสดงเฟรมสด (ไม่บล็อก)
    cv2.imshow("ALPR Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("ออกโดยผู้ใช้")
        break

    # ตรวจในช่วง interval เท่านั้น
    if now - last_check < CHECK_INTERVAL:
        continue
    last_check = now
    check_dt = datetime.now()
    print(f"\n--- ตรวจหา plate เวลา: {check_dt.isoformat()} ---")

    # ใช้ YOLO detect plates (ถ้าไม่มี plate จะได้ boxes เป็น empty)
    results = model.predict(frame, conf=0.4, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy() if len(results) > 0 else []

    detected_plate = None
    detected_province = None

    # หากเจอกล่องหลายกล่อง ให้พยายามอ่านทุกกล่องแล้วเลือกตัวแรกที่อ่านได้
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        h_box = y2 - y1
        extra = int(h_box * 0.6)
        y2_expanded = min(frame.shape[0], y2 + extra)
        crop = safe_crop(frame, x1, y1, x2, y2_expanded, pad=6)
        if crop.size == 0:
            continue

        # upscale + enhance
        crop_up = upscale_image(crop, scale=2)
        crop_enh = enhance_for_ocr(crop_up)

        # OCR via EasyOCR_ALPR
        ocr_results = ocr_engine.predict(crop_enh)
        texts = [r.text.strip() for r in ocr_results if hasattr(r, "text") and r.text.strip()]
        combined_text = " ".join(texts)

        plate_results = extract_thai_license_plate(combined_text)
        if plate_results:
            plate_info = plate_results[0]
            detected_plate = plate_info.get('plate')
            detected_province = plate_info.get('province') or extract_province_from_text(combined_text)
            print(f"  -> อ่านทะเบียนได้: {detected_plate} {detected_province or ''}")
            break

        else:
            # ถ้ายังอ่านไม่ได้ อาจลองใช้ combined_text เพื่อหา province หรือคำอื่น (ไม่จำเป็นที่นี่)
            continue

    # กรณีไม่มี plate ในรอบนี้
    if detected_plate is None:
        print("  ไม่มีป้ายรถที่อ่านได้ในรอบนี้")
        # ถ้าไม่มีรถ: ถ้ามี target เดิมไปเรื่อย ๆ ให้แต่ละ policy รีเซ็ตหรือเก็บ last_seen?
        # ตาม request: เมื่อไม่มีรถก็ไม่ทำอะไร (แต่ถ้าต้องการให้ target expire ให้เพิ่มเงื่อนไข)
        continue

    # ถ้าตอนนี้ยังไม่มี target -> ตั้ง target เป็น detected_plate
    if target_plate is None:
        target_plate = detected_plate
        target_first_seen = check_dt
        target_last_seen = check_dt
        print(f"  ตั้ง target ใหม่: {target_plate} เวลาเริ่ม {target_first_seen.isoformat()}")
        continue

    # ถ้ามี target อยู่แล้ว -> เปรียบเทียบกับ detected_plate
    if detected_plate != target_plate:
        # ถ้าไม่ตรง ให้รีเซ็ต target เป็น detected_plate ใหม่ (ตามที่ขอ)
        print(f"  พบทะเบียนใหม่ ({detected_plate}) แตกต่างจาก target เดิม ({target_plate}) -> รีเซ็ต target")
        target_plate = detected_plate
        target_first_seen = check_dt
        target_last_seen = check_dt
        continue
    else:
        # ถ้าตรงกัน (same plate) -> update last_seen และตรวจเวลา
        target_last_seen = check_dt
        elapsed = (target_last_seen - target_first_seen).total_seconds()
        print(f"  พบ target เดิม {target_plate} อีกครั้ง (elapsed = {elapsed:.1f} s)")

        if elapsed >= LONG_STAY_THRESHOLD:
            # ถ้าซ้ำเกิน threshold -> แคปภาพทั้งเฟรม (หรือ crop) + พิมพ์บันทึก
            fname = f"{OUTPUT_DIR}/{target_plate}_{target_last_seen.strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(fname, frame)
            print(f"\n>>> LONG STAY DETECTED: {target_plate}")
            print(f"    - first_seen: {target_first_seen.isoformat()}")
            print(f"    - last_seen : {target_last_seen.isoformat()}")
            print(f"    - elapsed   : {elapsed:.1f} seconds")
            print(f"    - saved image: {fname}")
            print(f"    - (ในสถานการณ์จริง จะส่ง API หรือแจ้งเตือนไปยังเพื่อนที่รับผิดชอบได้ที่นี่)\n")

            # หลังจากแคปและพิมพ์แล้ว ให้รีเซ็ต target เพื่อเริ่มหารอบใหม่
            target_plate = None
            target_first_seen = None
            target_last_seen = None
            # และ continue loop (จะรอ next CHECK_INTERVAL)
            continue

# end main loop
cap.release()
cv2.destroyAllWindows()


✅ EasyOCR_ALPR loaded (GPU=False)


INFO:open_image_models.detection.core.yolo_v9.inference:Using ONNX Runtime with ['AzureExecutionProvider', 'CPUExecutionProvider'] provider(s)
INFO:open_image_models.detection.pipeline.license_plate:Initialized LicensePlateDetector with model C:\Users\Tan\.cache\open-image-models\yolo-v9-t-384-license-plate-end2end\yolo-v9-t-384-license-plates-end2end.onnx


✅ เริ่มระบบ (ตรวจทุก 10 วินาที). กด q เพื่อออก

--- ตรวจหา plate เวลา: 2025-10-17T14:28:35.522967 ---
  ไม่มีป้ายรถที่อ่านได้ในรอบนี้

--- ตรวจหา plate เวลา: 2025-10-17T14:28:37.485297 ---
  ไม่มีป้ายรถที่อ่านได้ในรอบนี้

--- ตรวจหา plate เวลา: 2025-10-17T14:28:39.490569 ---
  ไม่มีป้ายรถที่อ่านได้ในรอบนี้

--- ตรวจหา plate เวลา: 2025-10-17T14:28:41.491722 ---
  -> อ่านทะเบียนได้: กง154 ตาก
  ตั้ง target ใหม่: กง154 เวลาเริ่ม 2025-10-17T14:28:41.491722

--- ตรวจหา plate เวลา: 2025-10-17T14:28:44.589755 ---
  -> อ่านทะเบียนได้: กง154 น่าน
  พบ target เดิม กง154 อีกครั้ง (elapsed = 3.1 s)

--- ตรวจหา plate เวลา: 2025-10-17T14:28:46.970667 ---
  -> อ่านทะเบียนได้: กง54 ตาก
  พบทะเบียนใหม่ (กง54) แตกต่างจาก target เดิม (กง154) -> รีเซ็ต target

--- ตรวจหา plate เวลา: 2025-10-17T14:28:49.582799 ---
  -> อ่านทะเบียนได้: กง154 น่าน
  พบทะเบียนใหม่ (กง154) แตกต่างจาก target เดิม (กง54) -> รีเซ็ต target

--- ตรวจหา plate เวลา: 2025-10-17T14:28:51.909781 ---
  -> อ่านทะเบียนได้: กง154 ตาก
  พบ t